In [1]:
enable_wandb = True
if enable_wandb:
    import wandb

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\dpa13\_netrc


True

In [5]:
import os
import pdb
import pandas
import torch

print(torch.__version__)

2.4.0+cu124


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:,0],z[:,1], s=70, c=color, cmap="set2")
    plt.show()

def embedding_to_wandb(h, color, key="embeddding"):
    num_components = h.shape[-1]
    df = pandas.DataFrame(data.h.detach().cpu().numpy(), columns=[f'c_{i}' for i in range(num_components)])
    df["target"] = color.detach().cpu().numpy().astype("str")
    cols = df.columsn.tolist()
    df = df[cols[-1:] + cols[:-1]]
    wandb.log({key:df})

#### node classification with GNN

In [10]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='../datasets/cora', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}')
print('===========')
print(f'# graphs: {len(dataset)}')
print(f'# features: {dataset.num_features}')
print(f'# classes: {dataset.num_classes}')



Dataset: Cora()
# graphs: 1
# features: 1433
# classes: 7


In [11]:
data = dataset[0]

print()
print(data)
print('=========')

# gather info
print(f'# nodes: {data.num_nodes}')
print(f'# edges: {data.num_edges}')
print(f'avg node degree: {data.num_edges/data.num_nodes:.2f}')
print(f'# training nodes: {data.train_mask.sum()}')
print(f'training node label rate: {int(data.train_mask.sum())/data.num_nodes:.2f}')
print(f'has isolated nodes: {data.has_isolated_nodes()}')
print(f'has self-loops: {data.has_self_loops()}')
print(f'is undirected: {data.is_undirected()}')


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
# nodes: 2708
# edges: 10556
avg node degree: 3.90
# training nodes: 140
training node label rate: 0.05
has isolated nodes: False
has self-loops: False
is undirected: True


In [14]:
import torch 
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)
    
    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)
        

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


In [15]:
if enable_wandb:
    wandb.init(project='node-classification')
    summary = dict()
    summary["data"] = dict()
    summary["data"]["num_features"] = dataset.num_features
    summary["data"]["num_classes"] = dataset.num_classes
    summary["data"]["num_nodes"] = data.num_nodes
    summary["data"]["num_edges"] = data.num_edges 
    summary["data"]["has_isolated_nodes"] = data.has_isolated_nodes()
    summary["data"]["has_self_nodes"] = data.has_self_loops()
    summary["data"]["is_undirected"] = data.is_undirected()
    summary["data"]["num_training_nodes"] = data.train_mask.sum()
    wandb.summary = summary

In [ ]:
model = MLP(hidden_channels=16)

